<a href="https://colab.research.google.com/github/sujnkim/Algorithm-study/blob/main/resolve/ch12%EA%B5%AC%ED%98%84%EB%B3%B5%EC%8A%B5(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 12-11: 뱀
문제 링크: https://www.acmicpc.net/problem/3190

**INPUT:**
* 첫째 줄에 보드의 크기 n (2<=n<=100)
* 둘째 줄에 사과의 개수 k (0<=k<=100)
* 다음 k개의 줄에 사과의 위치 행, 열
* 다음 줄에 뱀의 방향 변환 횟수 l (1<=l<=100)
* 다음 l개의 줄에 방향 변환 정보 정수 x, d
    * 시작 후 x초 뒤에 왼쪽(L), 오른쪽(D)로 90도 회전

**OUTPUT:**
* 게임이 몇 초에 끝나는지 출력

**요구사항:**
* 시작 위치는 (1,1), 뱀의 길이는 1, 처음에는 오른쪽을 향함
* 이동 규칙
    * 몸길이를 늘려 머리를 다음 칸에 위치
    * 이동한 칸에 사과가 있다면, 사과는 없어지고 꼬리는 움직이지 않음(몸 길이 1 증가)
    * 이동한 칸에 사과가 없다면, 꼬리가 위치한 칸을 비워줌(몸길이 변화 없음)
* 벽 또는 자기 자신의 몸과 부딪히면 게임이 종료

In [ ]:
# 예시1 => 9

n = 6
k = 3
data = [ (3,4), (2,5), (5,3) ]
l = 3
move = [ (3,'D'), (15,'L'), (17,'D') ]


In [ ]:
# 예시2 => 21

n = 10
k = 4
data = [ (1,2), (1,3), (1,4), (1,5) ]
l = 4
move = [ (8,'D'), (10,'D'), (11,'D'), (13,'L')]

In [ ]:
# 예시3 => 13

n = 10
k = 5
data = [ (1,5), (1,3), (1,2), (1,6), (1,7) ]
l = 4
move = [ (8,'D'), (10,'D'), (11,'D'), (13,'L')]

In [ ]:
from collections import deque

def solution():

    #사과 위치를 저장할 이차원 리스트
    apple = [ [0]*(n+1) for _ in range(n+1) ]    

    # 사과 위치 표시
    for d in data:
        x, y = d
        apple[x][y] = 1

    # 방향전환 시간 정보 저장
    change = []
    for i in range(l):
        change.append(move[i][0])

    # 북, 동, 남, 서 (오른쪽 회전)
    dx = [-1, 0, 1, 0]
    dy = [0, 1, 0, -1]

    #시작 위치
    x, y = 1, 1
    snake = deque([(x,y)])
    dir = 1
    time = 0


    while True:

        # 방향 전환
        if time in change:
            ndir = move[change.index(time)][1]
            #print(time, ndir)
            if ndir == 'L':
                dir = (dir+4-1) % 4
            elif ndir == 'D':
                dir = (dir+4+1) % 4
            #print(dir)

        # 다음 칸 위치
        nx = x + dx[dir]
        ny = y + dy[dir]
        time += 1

        # 종료 조건 확인
        if nx<=0 or ny<=0 or nx>n or ny>n:
            break
        if (nx,ny) in snake:
            break

        snake.append((nx,ny))

        if apple[nx][ny] == 1:    #사과가 있다면->사과제거
            apple[nx][ny] = 0
        else:                   #사과가 없다면->꼬리제거
            snake.popleft()

        #print(snake)
        x, y = nx, ny
            
    return time

solution()

13

### 12-12: 기둥과 보 설치
문제 링크: https://programmers.co.kr/learn/courses/30/lessons/60061

**INPUT:**
* 벽면의 크기n (5<=n<=100)
* 기둥과 보를 설치/삭제하는 작업이 순서대로 담긴 2차원 배열 build_frame
* [x,y,a,b] 형식
* a는 구조물 종류(0:기둥, 1:보)
* b는 작업 종류 (0:삭제, 1:설치)

**OUTPUT:**
* 명령어를 수행한 후 구조물의 상태를 return하는 solution 함수
* [x,y,a] 형식
* a는 구조물의 종류 (0:기둥, 1:보)
* x 좌표 기준으로 오름차순 정렬, x좌표가 같을 경우 y좌표 기준 오름차순 정렬

**요구사항:**
* 기둥은 바닥 위에 있거나, 보의 한쪽 끝부분 위에 있거나, 다른 기둥 위에 위치
* 보는 한쪽 끝부분이 기둥 위에 있거나, 양쪽 끝부분이 다른 보와 동시에 연결
* 기둥과 보를 설치/삭제하는 과정에서 항상 위의 규칙을 만족시켜야 함
* 만족시키지 않을 경우 해당 작업을 무시


**해결방법:**
* 일단 설치/삭제를 수행 후 확인 작업을 수행
    * 규칙을 만족시킨 상태인지 확인하는 함수 따로 구현
    * 규칙 만족 -> 진행 / 규칙 불만족 -> 원상복구


In [ ]:
# 현재 상태가 규칙을 만족하는지 확인하는 함수
# 만족하면 True, 만족하지 않으면 False 반환

def check(answer):

    for x, y, struct in answer:

        if struct == 0:     #기둥
            # 바닥 or 다른 기둥 위 or 한쪽 보 위(2방향)
            if y==0 or [x,y-1,0] in answer \
                    or [x-1,y,1] in answer or [x,y,1] in answer:
                continue
            else:
                return False


        if struct == 1:     #보
            #한쪽 끝이 기둥 위(2가지) or 양쪽이 다른 보와 연결(양쪽비교)
            if [x,y-1,0] in answer or [x+1,y-1,0] in answer \
                or ([x-1,y,1] in answer and [x+1,y,1] in answer):
                continue
            else:
                return False

    return True

In [ ]:
def solution(n, build_frame):
    answer = []


    for build in build_frame:
        x, y, struct, do = build
        
        if do ==1:     #설치명령
            answer.append([x,y,struct])     #일단 설치

            if check(answer) == False:
                answer.remove([x,y,struct]) # 규칙위반->삭제


        if do == 0:     #삭제명령
            answer.remove([x,y,struct])     #일단 삭제

            if check(answer) == False:
                answer.append([x,y,struct]) #규칙위반->재설치

    #결과 정렬
    answer.sort()

    return answer

In [ ]:
#예시1

n = 5
build_frame = [
    [1,0,0,1], [1,1,1,1], [2,1,0,1],
    [2,2,1,1], [5,0,0,1], [5,1,0,1],
    [4,2,1,1], [3,2,1,1]
]

print(solution(n, build_frame))

[[1, 0, 0], [1, 1, 1], [2, 1, 0], [2, 2, 1], [3, 2, 1], [4, 2, 1], [5, 0, 0], [5, 1, 0]]


In [ ]:
#예시2

n = 5
build_frame = [
    [0,0,0,1], [2,0,0,1], [4,0,0,1],
    [0,1,1,1], [1,1,1,1], [2,1,1,1], [3,1,1,1],
    [2,0,0,0], [1,1,1,0], [2,2,0,1]
]

print(solution(n, build_frame))

[[0, 0, 0], [0, 1, 1], [1, 1, 1], [2, 1, 1], [3, 1, 1], [4, 0, 0]]


### 12-13: 치킨 배달
문제 링크: https://www.acmicpc.net/problem/15686

**INPUT:**
* 첫째 줄에 도시 크기 n(2<=n<=50), 선택할 치킨집 개수 m(1<=m<=13)
* 둘째 줄부터 n개의 줄에 도시의 정보
    * 0: 빈칸, 1: 집, 2: 치킨집
    * 1<= 치킨집 개수<=2n

**OUTPUT:**
* 폐업시키지 않을 치킨집을 최대 m개 골랐을 때, 도시의 치킨 거리 최솟값 출력

**요구사항:**
* 도시의 칸은 (r,c)로 표현
* r, c는 1부터 시작
* 치킨거리 = 집과 가장 가까운 치킨집 사이의 거리
* 도시의 치킨 거리 = 모든 집의 치킨 거리의 합
* 거리는 각 좌표 차이의 절대값의 합
    * (r1, c1)과 (r2,c2)의 거리 = |r1-r2| + |c1-c2|



**해결방법:**
* 전체 치킨집 중 m개를 골라 도시의 치킨 거리를 계산해야 함
* 전체 중 m개를 고르는 경우의 수 => combination


In [ ]:
from itertools import combinations

def solution(n, m, data):
    
    chicken = []
    house = []

    # 집, 치킨집 좌표 저장 리스트
    for i in range(n):
        for j in range(n):
            if data[i][j] == 2:
                chicken.append((i,j))
            if data[i][j] == 1:
                house.append((i,j))
    
    #solve
    result = 1e9    # 도시 치킨 거리의 최소값


    # m개의 치킨집 조합에 대해 도시 치킨 거리 계산
    for selected in combinations(chicken, m):

        city_dist = 0

        # 각 집에 대하여 치킨 거리 계산
        for r1, c1 in house:
            chicken_dist = 2*n
            for r2, c2 in selected:     #각 집의 치킨 거리 계산
                chicken_dist = min(chicken_dist, abs(r1-r2)+abs(c1-c2) )
        
            city_dist += chicken_dist


        result = min(result, city_dist)

    
    return result

In [ ]:
#예시1

n, m = 5, 3
data = [
    [0,0,1,0,0],
    [0,0,2,0,1],
    [0,1,2,0,0],
    [0,0,1,0,0],
    [0,0,0,0,2]
]

print(solution(n,m,data))

5


In [ ]:
#예시2

n, m = 5, 2
data = [
    [0,2,0,1,0],
    [1,0,1,0,0],
    [0,0,0,0,0],
    [2,0,0,1,1],
    [2,2,0,1,2]
]

print(solution(n,m,data))

10


In [ ]:
#예시3

n, m = 5, 1
data = [
    [1,2,0,0,0],
    [1,2,0,0,0],
    [1,2,0,0,0],
    [1,2,0,0,0],
    [1,2,0,0,0]
]

print(solution(n,m,data))

11


### 12-14: 외벽 점검
문제 링크: https://programmers.co.kr/learn/courses/30/lessons/60062


**INPUT:**
* 외벽의 길이 n (1<=n<=200)
* 취약 지점 위치가 담긴 배열 weak (1<=weak의 길이<=15)
    * 서로 다른 두 취약점의 위치는 같지 않음
    * 취약 지점 위치는 오름차순으로 정렬되어 있음
    * weak의 원소는 0이상 n-1이하 정수
* 각 친구가 1시간 동안 이동할 수 있는 거리가 담긴 배열 dist
    * 1 <= dist길이 <= 8
    * dist의 원소는 1이상 100이하 정수


**OUTPUT:**
* 취약 지점을 점검하기 위해 보내야하는 친구 수의 최솟값을 return하는 solution함수 완성
* 친구를 모두 투입해도 취약 지점을 전부 점검할 수 없는 경우 -1 return

**요구사항:**
* 레스토랑 구조는 원형
* 정북 방향을 0으로 설정
* 취약 지점 위치는 정북 방향을 기준으로 시계방향으로 떨어진 거리를 의미
* 친구들은 출발 지점에서 시계 or 반시계 방향으로 외벽을 따라 이동

In [2]:
from itertools import permutations

def solution(n, weak, dist):
    
    # 길이를 2배로 늘려 원형을 일자로 표현
    length = len(weak)
    for i in range(length):
        weak.append(weak[i]+n)


    result = len(dist) + 1      # 최대 인원수 + 1로 초기화

    # 각 weak 지점을 시작점으로 설정
    for start in range(length):

        #친구를 나열하는 모든 경우에 수에 대해 확인
        for friends in list(permutations(dist, len(dist))):
            count = 1

            #해당 친구가 점검할 수 있는 마지막 위치
            position = weak[start] + friends[count-1]

            #시작점부터 모든 취약점 확인
            for index in range(start, start+length):
                #점검 위치를 벗어나는 경우
                if position < weak[index]:
                    #친구 추가
                    count += 1
                    
                    if count > len(dist):   #투입이 불가능한 경우, 종료
                        break

                    position = weak[index] + friends[count-1]

            result = min(result, count)

    if result > len(dist):
        return -1

    return result

In [3]:
#예시1

n=12
weak = [1,5,6,10]
dist = [1,2,3,4]

print(solution(n, weak, dist))

2


In [4]:
#예시2

n=12
weak = [1,3,4,9,10]
dist = [3,5,7]

print(solution(n, weak, dist))

1
